# Transfer Learning with TensorFlow Part 1: Feature Extraction

Transfer Learning is leveraging a working model's existing architecture and learned patterns for our own problem.

There are two mein benefits:
1. Can leverage an existing neural network architecture proven to work on problem similar to our own.
2. Can leverage a working neural network architecture which has already learning patterns on similar data to our own, then we can adapt those pattern to our own data.

In [1]:
# Are we using a GPU?
!nvidia-smi

Thu Jun 23 05:42:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.40       Driver Version: 516.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   56C    P5    19W /  N/A |   1354MiB /  6144MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download and becoming one with the data

In [11]:
# Let's get data (10% of 10 food classes from Food101)
food_dir = "10_food_classes_10_percent/"

In [12]:
# How many images in each folder
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk(food_dir):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

There are 2 directories and 1 images in '10_food_classes_10_percent/'
There are 10 directories and 0 images in '10_food_classes_10_percent/test'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\chicken_curry'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\chicken_wings'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\fried_rice'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\grilled_salmon'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\hamburger'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\ice_cream'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\pizza'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\ramen'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\steak'
There are 0 directories and 250 images in '10_food_classes_10_percent/test\sush

## Creating data loaders (preparing the data)

We'll use the `ImageDataGenerator` class to load in our images as batches.

In [13]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir = food_dir + "train"
test_dir = food_dir + "test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images: ")
train_data_10_percent = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("Testing images: ")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Training images: 
Found 750 images belonging to 10 classes.
Testing images: 
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before training too long and it overfits) with the EarlyStopping callback

In [ ]:
# Create TensorBoard callback (functionized because we need to create a new one for each model)
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

## Creating model using TensorFlow Hub

In the past we've used TensorFlow to create our own models layer by layer from scratch.

Now we're going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.

We can access pretrained models on: https://www.tensorflow.org/hub/

Browsing the TensorFlow Hub page and sorting for image classification, we found the following feature vector model link: https://tfhub.dev/google/efficientnet/b0/feature-vector/1

In [14]:
import tensorflow_hub as hub